### Imports

In [1]:
import tensorflow as tf

from data_generator import DataGenerator
from train_test_utils import model_evaluate, model_fit
from model import ASRModel

import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.0.0


In [3]:
tf.test.is_gpu_available()

True

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Paths

In [5]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

### Create DataGenerator objects

In [6]:
train_data = DataGenerator(train_path)
val_data = DataGenerator(dev_path)
test_data = DataGenerator(test_path)

### Build Model

In [7]:
model = ASRModel()
# model.build(input_shape = (None, None, 20))
optimizer = tf.keras.optimizers.Adam()

In [8]:
# model.summary()

### Checkpoint

In [9]:
ckpt_dir = './training_checkpoints'
ckpt = tf.train.Checkpoint(epoch=tf.Variable(1), optimizer=optimizer, model = model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep = 2)    

In [ ]:
losses, accuracies, val_losses, val_acc = model_fit(model, optimizer, train_data, manager, ckpt, val_ds = val_data, epochs = 20)

Restored from ./training_checkpoints/ckpt-85
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
78
156
234
312
390
468
546
Saved checkpoint for epoch 75: ./training_checkpoints/ckpt-86
Epoch:  75  Loss: 394.674164  WER:  0.9998624250136957
 Validation Loss: 246.747086  Validation WER:  0.9989873015992004
78
156
234
312
390
468
546
Saved checkpoint for epoch 76: ./training_checkpoints/ckpt-87
Epoch:  76  Loss: 394.488617  WER:  1.0000055755466082
 Validation Loss: 246.646988  Validation WER:  0.9994585712611774
78
156
234
312
390
468
546
Saved checkpoint for epoch 77: ./training_checkpoints/ckpt-88
Epoch:  77  Loss: 394.305176  WER:  0.9998472842321625
 Validation Loss: 246.555756  Validation WER:  0.9996015169323982
78
156
234
312


In [ ]:
def plot(parameter, name):
    plt.plot(parameter)
    plt.ylabel(name)
    plt.xlabel('iterations')
    plt.show()

In [ ]:
plot(losses, "Training Loss")
plot(accuracies, "Training WER")
plot(val_losses, "Validation Loss")
plot(val_acc, "Validation WER")

### Restore Checkpoint and Test Model

In [ ]:
save_file = open('outputs/predictions.txt', 'w')

In [ ]:
ckpt.restore(manager.latest_checkpoint)

_, acc = model_evaluate(model, test_data, test=True, save_file = save_file)

In [ ]:
print(acc)